# Board Game Recommender

### Import libraries and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
games_df_full = pd.read_csv('./data/clean/games_df.csv', index_col=0)

c:\programdata\miniconda3\envs\newenv\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
games_df_full.head(1)

,user,rating,comment,ID,name,description,minplayers,maxplayers,playingtime,minage,usersrated,average
360434,166554,6.0,NaN,68448,7 Wonders,You are the leader of one of the 7 great citie...,2,7,30,10,79873,7.76049


In [18]:
# subset of full dataset for recommender
games_df = games_df_full[['name', 'ID', 'user', 'rating']]

In [19]:
#create a sample dataframe with less entries, sample 100,000 reviews
games_df_subset = games_df.sample(n=100000, replace=False)
games_df_subset.shape

(100000, 4)

### Prepare recommender using cosine distances

In [20]:
#pivot table from the books_df dataframe
pivot_df = pd.pivot_table(games_df_subset, index='name', columns='user', values='rating')
#create sparse matrix
sparse_df = sparse.csr_matrix(pivot_df.fillna(0))
#calculate cosine distances for similarities
recommender = pairwise_distances(sparse_df, metric='cosine')
#export as a dataframe
rec_df = pd.DataFrame(recommender, columns=pivot_df.index, index=pivot_df.index)

In [21]:
# get average rating, number of players, playing time, etc to filter by in app, group by game ID
games_info = games_df_full.groupby('name').mean()
games_info.drop(columns=['user'], inplace=True)
games_info.rename(columns={'rating': 'average_rating'}, inplace=True)
games_info.head(2)

rec_df = pd.merge(left = rec_df, right= games_info, right_index = True, left_index = True)

### Define book Recommender function

In [22]:
rec_df

,"""Oh My God! There's An Axe In My Head."" The Game of International Diplomacy",'65: Squad-Level Combat in the Jungles of Vietnam,"...and then, we held hands.",...und tschüss!,011,1 Stein + Co.,"1,000 Places to See Before You Die","1,2,3! Now you see me...",10 Days in Africa,10 Days in Asia,...,なつめも (Natsumemo),ひとひら (Hitohira),average_rating,ID,minplayers,maxplayers,playingtime,minage,usersrated,average
name,,,,,,,,,,,,,,,,,,,,,
"""Oh My God! There's An Axe In My Head."" The Game of International Diplomacy",0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,5.696000,23304.0,2.0,6.0,60.0,16.0,79.0,5.90557
'65: Squad-Level Combat in the Jungles of Vietnam,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,7.802041,188325.0,2.0,3.0,180.0,12.0,134.0,7.70112
"...and then, we held hands.",1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,6.053841,153999.0,2.0,2.0,45.0,12.0,1767.0,6.11144
...und tschüss!,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,6.520745,853.0,4.0,6.0,30.0,10.0,322.0,6.41807
011,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,6.126838,93194.0,3.0,6.0,90.0,12.0,414.0,6.10952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Свинтус,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,5.380800,46147.0,2.0,8.0,30.0,6.0,164.0,5.45268
Экивоки,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,6.715789,297864.0,2.0,16.0,60.0,16.0,37.0,6.73243
€uro Crisis,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,7.061290,165022.0,3.0,4.0,90.0,14.0,95.0,7.01905


In [78]:
games_df_full[games_df_full['name'] =='Dominion: Intrigue'][:1]

,user,rating,comment,ID,name,description,minplayers,maxplayers,playingtime,minage,usersrated,average
2444546,179379,6.5,NaN,40834,Dominion: Intrigue,"In Dominion: Intrigue (as with Dominion), each...",2,4,30,13,29221,7.7271


In [79]:
games_df_full[games_df_full['name'] =='Roll for the Galaxy'][:1]

,user,rating,comment,ID,name,description,minplayers,maxplayers,playingtime,minage,usersrated,average
3154527,159712,8.0,NaN,132531,Roll for the Galaxy,Game description from the publisher:&#10;&#10;...,2,5,45,13,24804,7.6873


In [96]:
def game_recommender(game_title, average_rating = 7.5, minplayers = -np.inf, playingtime = np.inf):
    rec_df2 = rec_df[rec_df['average_rating'] >= average_rating]
    rec_df3 = rec_df2[rec_df2['minplayers'] >= minplayers]
    rec_df4 = rec_df3[rec_df3['playingtime'] <= playingtime]
    return 1- rec_df4[game_title].sort_values()[1:6]

In [98]:
game_recommender('Cosmic Encounter')

name
Star Trek: Attack Wing           0.032615
The Lord of the Ice Garden       0.032298
Black Sonata                     0.025629
Cataclysm: A Second World War    0.023602
Wilderness War                   0.022686
Name: Cosmic Encounter, dtype: float64

In [102]:
#add game title, re-arrange for streamlit app
rec_df.insert(0, "game_title", rec_df.index)

# save rec_df for use in streamlit app
rec_df.to_pickle('./streamlit_app/data/games.pk1')

In [107]:
rec_df['playingtime'].max()

22500.0